# Single Barrier Option Pricing & Applications to Leveraged (Turbo) Certificates

## Introduction

This notebook contains the valuation functions for single barrier options within the Black and Scholes (1973) framework. The risk-neutral dynamics of the underlying asset are given by

\begin{equation*}
\mathrm{d}S_t = (r - \delta) S_t \mathrm{d}t + \sigma S_t \mathrm{d}W_t.
\end{equation*}

I employ the method of images applied to one-dimensional power binary options as the basic building blocks. The main references in the context of single barrier options are Wilmott et al. (1995), Buchen (2001) and Konstandatos (2003).

The notation in this notebook closely follows the one in my Ph.D. dissertation Thul (2013) and the paper Zhang and Thul (2017).

## References

Buchen, Peter W. (2001) "Image Options and the Road to Barriers," *Risk Magazine*, Vol. 14, No. 9, pp. 127-130

Konstandatos, Otto (2003) "A New Framework for Pricing Barrier and Lookback Options," Ph.D. Dissertation, University of Sydney, 

Thul, Matthias (2013) "Characterizations of and Closed-Form Solutions for Plain Vanilla and Exotic Derivatives," Ph.D. Dissertation, University of New South Wales, [available online](http://handle.unsw.edu.au/1959.4/57323)

Zhang, Ally Quan and Matthias Thul (2017) "How Much is the Gap? Efficient Jump-Risk Adjusted Valuation of Leverage Certificates," *Quantitative Finance*, forthcoming, [available on SSRN](https://ssrn.com/abstract=2388734)

Wilmott, Paul, Sam Howison, and Jeff Dewynne (1995) *The Mathematics of Financial Derivatives*: Cambridge University Press

In [32]:
import numpy
import scipy.stats as stats

## First Order Binary Option Pricing Functions

The payoff of a first order power binary option with exponent $\eta \in \mathbb{R}$ is given by

\begin{equation*}
{^\eta \mathcal{P}_\xi^s(S, 0)} = S^\eta \mathrm{1} \{ s S > s \xi \}.
\end{equation*}

Here, $\xi \in \mathbb{R}_+$ is the strike price and $s \in \{ -1, +1 \}$ indicates a put or call option, respectively. Its valuation function is given by

\begin{equation*}
{^\eta \mathcal{P}_\xi^s(S, \tau)} = F_\eta \mathcal{N} \left( s d_\eta \right),
\end{equation*}

where

\begin{eqnarray*}
F_\eta & = & S^\tau \exp \left\{ \left( (\eta - 1) \left( r + \frac{1}{2} \eta \sigma^2 \right) - \eta \delta \right) \tau \right\}\\
d_\eta & = & \frac{1}{\sigma \sqrt{\tau}} \left( \ln \left( \frac{S}{\xi} \right) + \left( r - \delta + \left( \eta - \frac{1}{2} \right) \sigma^2 \right) \tau \right).
\end{eqnarray*}

These are special cases of Definition B.1 and Proposition B.1 in Zhang and Thul (2017). We refer to the special cases of $\eta = 1$ and $\eta = 0$ as asset and bond binary options. The delta is given by

\begin{equation*}
\frac{\partial}{\partial S} {^\eta \mathcal{P}_\xi^s(S, \tau)} = \frac{F_\eta}{S} \left( \mathcal{N} \left( s d_\eta \right) \mathrm{1} \{ \eta \neq 0 \} + \frac{s}{\sigma \sqrt{\tau}} \mathcal{N}' \left( s d_\eta \right) \right).
\end{equation*}

In [102]:
def powerBinary(maturity, eta, xi, phi, spot, rate, dividend, volatility):
    totalVolatility = volatility * numpy.sqrt(maturity)
    dValue = (numpy.log(spot / xi) + (rate - dividend + (eta - 0.5) * volatility**2) * maturity) / totalVolatility
    dCDF = stats.norm.cdf(phi * dValue)
    discountedForward = numpy.power(spot, eta) * numpy.exp(((eta - 1.0) * (rate + 0.5 * eta * volatility**2) - eta * dividend) * maturity)
    value = discountedForward * dCDF
    derivative = discountedForward / spot * (dCDF * (eta != 0.0) + phi * stats.norm.pdf(phi * dValue) / totalVolatility)
    return (value, derivative)

def assetBinary(maturity, xi, phi, spot, rate, dividend, volatility):
    return powerBinary(maturity, 1.0, xi, phi, spot, rate, dividend, volatility)

def bondBinary(maturity, xi, phi, spot, rate, dividend, volatility):
    return powerBinary(maturity, 0.0, xi, phi, spot, rate, dividend, volatility)

# Image Operator

The image of the valuation function $\tilde{V}(S, \tau)$ around the barrier $B \in \mathbb{R}_+$ is defined as

\begin{equation*}
\stackrel{B}{\mathcal{I}} \left\{ \tilde{V}(S, \tau) \right\} = \left( \frac{S}{B} \right)^{2 \alpha} \tilde{V} \left( \frac{B^2}{S}, \tau \right),
\end{equation*}

where

\begin{equation*}
\alpha = \frac{1}{2} - \frac{r - \delta}{\sigma^2}.
\end{equation*}

The delta of the image operator is given by

\begin{equation*}
\frac{\partial}{\partial S} \stackrel{B}{\mathcal{I}} \left\{ \tilde{V}(S, \tau) \right\} = \left( \frac{S}{B} \right)^{2 \alpha} \left( \frac{2 \alpha}{S} \tilde{V} \left( \frac{B^2}{S}, \tau \right) - \frac{B^2}{S^2} \frac{\partial \tilde{V}}{\partial S} \left( \frac{B^2}{S}, \tau \right) \right).
\end{equation*}

In [98]:
def imageOperator(valueFunction, barrier, spot, rate, dividend, volatility):
    alpha = 0.5 - (rate - dividend) / volatility**2
    imageFactor = (spot / barrier)**(2.0 * alpha)
    value, derivative = valueFunction(barrier**2 / spot)
    value2 = imageFactor * value
    derivative2 = imageFactor * (2.0 * alpha / spot * value - (barrier / spot)**2 * derivative)
    return (value2, derivative2)

## Pay-at-Hit Rebate Valuation

A pay-at-hit rebate option has a unit payoff upon the first hitting time of $S$ to some barrier $B \in \mathbb{R}_+$. Its valuation function is given by

\begin{equation*}
\mathcal{R}_B^\psi(S, \tau) = \mathcal{U}_B^\psi(S) - B^{-\beta(-\psi)} \left( {^{\beta(-\psi)} \mathcal{P}_B^\psi} (S, \tau) - \stackrel{B}{\mathcal{I}} \left\{ {^{\beta(-\psi)} \mathcal{P}_B^\psi} (S, \tau) \right\} \right),
\end{equation*}

where

\begin{eqnarray*}
\mathcal{U}_B^\psi(S) & = & \left( \frac{S}{B} \right)^{\beta(-\psi)}, \\
\beta(\psi) & = & \alpha + \psi \sqrt{\lambda + \alpha^2},\\
\lambda & = & \frac{2 r}{\sigma^2}
\end{eqnarray*}

and $\psi \in \{ -1, +1 \}$ indicates an upper or lower barrier, respectively; see Definition C.2 and Proposition C.2 in Zhang and Thul (2017).

Note that we need to check that the term under the square root in the expression for $\beta$ is non-negative. This can arise when e.g. the interest rate is sufficiently negative. See e.g. Le Floc'h (2015) for remedies.

## References

Le Floc'h, Fabien (2015) "Barrier Options Under Negative Rates in Black-Scholes," Working Paper

In [99]:
def payAtHitRebate(maturity, barrier, psi, spot, rate, dividend, volatility):
    if (psi * spot <= psi * barrier):
        return (1.0, 0.0)
    alpha = 0.5 - (rate - dividend) / volatility**2
    if (2.0 * rate / volatility**2 + alpha**2 < 0.0):
        raise Exception("unsupported rate/dividend/volatility combination")
    beta = alpha - psi * numpy.sqrt(2.0 * rate / volatility**2 + alpha**2)
    value = numpy.power(spot / barrier, beta)
    derivative = value / spot * beta
    valueFunction = lambda spot: powerBinary(maturity, beta, barrier, psi, spot, rate, dividend, volatility)
    value2, derivative2 = valueFunction(spot)
    value3, derivative3 = imageOperator(valueFunction, barrier, spot, rate, dividend, volatility)
    barrierPower = numpy.power(barrier, -beta)
    return (value - barrierPower * (value2 - value3), derivative - barrierPower * (derivative2 - derivative3))

## Leveraged Certificates

We give an example of how to use the above formulas to price a leveraged (turbo) certificate. These are essentially out-barrier options with an at- or in-the-money barrier. Their rebate is equal to the instrinsic value upon the knock-out. The valuation function is given by

\begin{eqnarray*}
\tilde{V}(S, \tau) & = & \phi \left( \mathcal{A}_B^\phi(S, \tau) - K \mathcal{B}_B^\phi(S, \tau) - \stackrel{B}{\mathcal{I}} \left\{ \mathcal{A}_B^\phi(S, \tau) - K \mathcal{B}_B^\phi(S, \tau) \right\} \right)\\
& & + \phi (B - K) \mathcal{R}_B^\phi(S, \tau);
\end{eqnarray*}

see Proposition 2.1 in Zhang and Thul (2017).

In [105]:
def leveragedCertificate(maturity, strike, barrier, phi, spot, rate, dividend, volatility):
    if (phi * strike > phi * barrier):
        raise Exception("barrier has to be at- or in-the-money")
    if (phi * spot <= phi * barrier):
        return (numpy.maximum(phi * (spot - strike), 0.0), 0.0)
    def valueFunction(spot):
        value, derivative = assetBinary(maturity, barrier, phi, spot, rate, dividend, volatility)
        value2, derivative2 = bondBinary(maturity, barrier, phi, spot, rate, dividend, volatility)
        return (value - strike * value2, derivative - strike * derivative2)
    value, derivative = valueFunction(spot)
    value2, derivative2 = imageOperator(valueFunction, barrier, spot, rate, dividend, volatility)
    value3, derivative3 = payAtHitRebate(maturity, barrier, phi, spot, rate, dividend, volatility)
    return (phi * (value - value2 + (barrier - strike) * value3), phi * (derivative - derivative2 + (barrier - strike) * derivative3))

## Unit Tests

For the special case when $r = \delta = 0$, the value of the leveraged certificate is always equal to the intrinsic value. We run a few tests to check this.

In [107]:
def testLeveragedCertificate():
    testCases = [{'barrier': 100.0, 'isCall': True, 'spots': [101.0, 102.0, 105.0, 110.0]},
                 {'barrier': 100.0, 'isCall': False, 'spots': [99.0, 98.0, 95.0, 90.0]},
                 {'barrier': 101.0, 'isCall': True, 'spots': [102.0, 105.0, 110.0]},
                 {'barrier': 99.0, 'isCall': False, 'spots': [98.0, 95.0, 90.0]}]
    for testCase in testCases:
        for spot in testCase['spots']:
            phi = 1.0 if testCase['isCall'] else -1.0
            value, derivative = leveragedCertificate(1.0, 100.0, testCase['barrier'], phi, spot, 0.0, 0.0, 0.2)
            numpy.testing.assert_almost_equal(value, phi * (spot - 100.0), 8)
            numpy.testing.assert_almost_equal(derivative, phi, 8)

testLeveragedCertificate()